Julien Gauthier

# Reinforcement Learning (Deep Q) Project

#### WARNING : install the correct versions of gym/tensorflow/keras-rl2 in a virtual environment.

In [ ]:
# These are the correct versions for this project to work :
%pip install tensorflow==2.12.0 keras-rl2==1.0.5 gym==0.25.2
%pip install pygame

### I. Setting up the OpenAI Cart Pole environment

In [2]:
import gym
import random

The environment is where the experiment takes place, the states are the different input parameters (in this case : cart position, cart velocity, pole angle, pole tip velocity) and the actions are the output possibilities (move the cart left or right).

In [10]:
env = gym.make("CartPole-v1", render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

Testing the environment with random actions.

In [39]:
episodes = 10
for episode in range(1, episodes + 1) :
    state = env.reset()
    score = 0
    done = False
    
    while not done :
        env.render()
        action = random.choice([0, 1])
        next_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\gym\core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode:1 Score:9.0
Episode:2 Score:19.0
Episode:3 Score:9.0
Episode:4 Score:10.0
Episode:5 Score:18.0
Episode:6 Score:15.0
Episode:7 Score:15.0
Episode:8 Score:16.0
Episode:9 Score:11.0
Episode:10 Score:11.0


### II. Deep Learning model with Keras.

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

Let's create a function that will build our model.

In [26]:
def build_model(states, actions) :
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

We can now use our function to create and show an instance of the model :

In [27]:
model = build_model(states, actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


### III. Agent creation with Keras-RL

In [28]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Let's create a function to build an agent with a given model and possible actions. We'll use the Boltzmann Q Policy and the DQN Algorithm.

In [29]:
def build_agent(model, actions) :
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

Now, let's build the agent

In [30]:
dqn = build_agent(model, actions)

### IV. Training (and visualizing) the agent

We can now train the agent : (visualize=True to see the progress in real time)

In [42]:
env = gym.make("CartPole-v1", render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=20000, visualize=True, verbose=1)

Training for 20000 steps ...
Interval 1 (0 steps performed)
   10/10000 [..............................] - ETA: 3:21 - reward: 1.0000

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   22/10000 [..............................] - ETA: 8:19 - reward: 1.0000

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated

   31/10000 [..............................] - ETA: 6:50 - reward: 1.0000

c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 22 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 23 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 24 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\Users\julie\OneDrive\Bureau\ENSEEIHT\projet-perso\reinforcement-learning-cart\.venv\Lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated

10000/10000 [==============================] - 207s 21ms/step - reward: 1.0000
100 episodes - episode_reward: 99.840 [9.000, 303.000] - loss: 2.095 - mae: 19.773 - mean_q: 40.177

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 205s 20ms/step - reward: 1.0000
done, took 412.779 seconds


Save your model before pygame crashes ! (TO DO : FIX)

In [43]:
dqn.save_weights('pre-trained-model-20ksteps.h5f', overwrite=True)

#### WARNING : this will reset your model ! (ONLY IF YOU WANT TO RE-TRAIN YOUR MODEL)

In [40]:
del model
del dqn
model = build_model(states, actions)
dqn = build_agent(model, actions)

# You can now re-run the previous cell to train the model

##### Test a saved model in the Cart Pole environment :

In [36]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

dqn.load_weights('pre-trained-model-20ksteps.h5f') # Load the pre-trained model from the repository

In [38]:
dqnscores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(dqnscores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 8.000, steps: 8
Episode 2: reward: 9.000, steps: 9
Episode 3: reward: 10.000, steps: 10
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 9.000, steps: 9
Episode 6: reward: 8.000, steps: 8
Episode 7: reward: 8.000, steps: 8
Episode 8: reward: 10.000, steps: 10
Episode 9: reward: 10.000, steps: 10
Episode 10: reward: 10.000, steps: 10
9.2
